In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
r_col = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('./data/u.data', sep='\t', names=r_col, usecols=range(3))

m_col = ['movie_id', 'title']
movies = pd.read_csv('./data/u.item', sep='|', names=m_col, usecols=range(2))

ratings = pd.merge(movies, ratings, on=['movie_id'])
display(ratings[:3])

,movie_id,title,user_id,rating
0,1,Toy Story (1995),308,4
1,1,Toy Story (1995),287,5
2,1,Toy Story (1995),148,4


In [3]:
movieRatings = ratings.pivot_table(index=['user_id'], columns=['title'], values='rating')
display(movieRatings[:5])

title,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown,� k�ldum klaka (Cold Fever) (1994)
user_id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
starWarsRatings = movieRatings['Star Wars (1977)']
starWarsRatings.head()

user_id
0    5.0
1    5.0
2    5.0
3    NaN
4    5.0
Name: Star Wars (1977), dtype: float64

In [5]:
similarity = movieRatings.corrwith(starWarsRatings)
similarity = similarity.dropna()
df = pd.DataFrame({
    'similarity': similarity
})
df.head()

/opt/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:2487: RuntimeWarning: Degrees of freedom <= 0 for slice
  warnings.warn("Degrees of freedom <= 0 for slice", RuntimeWarning)


,similarity
title,
'Til There Was You (1997),0.872872
1-900 (1994),-0.645497
101 Dalmatians (1996),0.211132
12 Angry Men (1957),0.184289
187 (1997),0.027398


In [6]:
sim = similarity.order(ascending=False)
display(sim[:5])

/opt/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: order is deprecated, use sort_values(...)
  if __name__ == '__main__':


title
Hollow Reed (1996)        1.0
Man of the Year (1995)    1.0
Stripes (1981)            1.0
Full Speed (1996)         1.0
Golden Earrings (1947)    1.0
dtype: float64

###### Need some confidence level for these recommendations

In [7]:
movieStats = ratings.groupby('title').agg({'rating': [np.size, np.mean]})
display(movieStats[:5])

rating          
                            size      mean
title                                     
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

In [8]:
popular = movieStats[(movieStats['rating']['size'] >= 100)].sort_values([('rating', 'mean')], ascending=False)
display(popular[:10])

rating          
                                   size      mean
title                                            
Close Shave, A (1995)               112  4.491071
Schindler's List (1993)             298  4.466443
Wrong Trousers, The (1993)          118  4.466102
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230
Rear Window (1954)                  209  4.387560
Usual Suspects, The (1995)          267  4.385768
Star Wars (1977)                    584  4.359589
12 Angry Men (1957)                 125  4.344000
Citizen Kane (1941)                 198  4.292929

In [9]:
df = popular.join(pd.DataFrame(similarity, columns=['similarity']))
display(df[:5])

/opt/anaconda2/lib/python2.7/site-packages/pandas/tools/merge.py:205: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


,"(rating, size)","(rating, mean)",similarity
title,,,
"Close Shave, A (1995)",112,4.491071,0.183451
Schindler's List (1993),298,4.466443,0.100933
"Wrong Trousers, The (1993)",118,4.466102,0.216204
Casablanca (1942),243,4.456790,0.248016
"Shawshank Redemption, The (1994)",283,4.445230,0.174986


In [10]:
df.sort_values(by='similarity', ascending=False, inplace=True)
display(df[:5])

,"(rating, size)","(rating, mean)",similarity
title,,,
Star Wars (1977),584,4.359589,1.000000
"Empire Strikes Back, The (1980)",368,4.206522,0.748353
Return of the Jedi (1983),507,4.007890,0.672556
Raiders of the Lost Ark (1981),420,4.252381,0.536117
Austin Powers: International Man of Mystery (1997),130,3.246154,0.377433


###### The filtering is just done with filter out unpopular movies